In [1]:
#https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
# Data should get from https://www.kaggle.com/c/bnp-paribas-cardif-claims-management/data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler


In [11]:
data = pd.read_csv('train.csv', nrows=50000)
data.shape
#data.head()

(50000, 133)

In [12]:
numerics = ['int16','int32','int64','float16','float32','float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

(50000, 114)

In [13]:
data[:10]

ID  target        v1        v2        v4         v5        v6        v7  \
0   3       1  1.335739  8.727474  3.921026   7.915266  2.599278  3.176895   
1   4       1       NaN       NaN       NaN   9.191265       NaN       NaN   
2   5       1  0.943877  5.310079  4.410969   5.326159  3.979592  3.928571   
3   6       1  0.797415  8.304757  4.225930  11.627438  2.097700  1.987549   
4   8       1       NaN       NaN       NaN        NaN       NaN       NaN   
5   9       0       NaN       NaN       NaN   8.856791       NaN       NaN   
6  12       0  0.899806  7.312995  3.494148   9.946200  1.926070  1.770427   
7  21       1       NaN       NaN       NaN        NaN       NaN       NaN   
8  22       0  2.078651  8.462619  3.739030   5.265636  1.573033  2.303371   
9  23       1  1.144802  5.880606  3.244469   9.538384  2.500001  1.559405   

         v8         v9  ...      v121      v122      v123      v124      v126  \
0  0.012941   9.999999  ...  0.803572  8.000000  1.989780  0.035754  1.804126   
1  2.301630        NaN  ...       NaN       NaN       NaN  0.598896       NaN   
2  0.019645  12.666667  ...  2.238806  9.333333  2.477596  0.013452  1.773709   
3  0.171947   8.965516  ...  1.956521  7.018256  1.812795  0.002267  1.415230   
4       NaN        NaN  ...       NaN       NaN       NaN       NaN       NaN   
5  0.359993        NaN  ...       NaN       NaN       NaN  0.049861       NaN   
6  0.066251   5.011287  ...  2.232558  3.476299  1.992594  0.083758  3.276100   
7       NaN        NaN  ...       NaN       NaN       NaN       NaN       NaN   
8  0.015869  11.111111  ...  1.276595  8.148148  1.875560  0.018659  1.159637   
9  0.412610   9.977529  ...  2.715964  7.325843  4.896617  0.008944  1.344550   

       v127      v128  v129      v130      v131  
0  3.113719  2.024285     0  0.636365  2.857144  
1       NaN  1.957825     0       NaN       NaN  
2  3.922193  1.120468     2  0.883118  1.176472  
3  2.954381  1.990847     1  1.677108  1.034483  
4       NaN       NaN     0       NaN       NaN  
5       NaN  1.536222     0       NaN       NaN  
6  1.623298  2.266575     0  2.263736  0.970873  
7       NaN       NaN     0       NaN       NaN  
8  5.582865  1.105283     0  1.170731  3.333334  
9  1.601176  1.928009     0  3.174603  1.000000  

[10 rows x 114 columns]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target', 'ID'], axis=1),
    data['target'],
    test_size=0.3,
    random_state=0)
X_train.shape, X_test.shape

((35000, 112), (15000, 112))

In [15]:
scaler = StandardScaler()
scaler.fit(X_train.fillna(0))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [17]:
# old sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
sel_.fit(scaler.transform(X_train.fillna(0)), y_train)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [18]:
sel_.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True, False,  True,
       False,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True,  True, False, False,  True,  True,
       False,  True, False,  True,  True,  True,  True,  True, False,
       False, False, False, False, False,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
       False, False, False,  True,  True,  True,  True, False, False,
       False, False,  True,  True,  True,  True,  True, False, False,
        True, False,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True, False, False,  True,  True, False,  True,
        True,  True,  True, False])

In [19]:
selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

total features: 112
selected features: 80
features with coefficients shrank to zero: 32


In [20]:
np.sum(sel_.estimator_.coef_ == 0)
removed_feats = X_train.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
removed_feats


Index(['v11', 'v20', 'v29', 'v33', 'v41', 'v42', 'v48', 'v49', 'v53', 'v55',
       'v62', 'v63', 'v64', 'v65', 'v67', 'v68', 'v81', 'v86', 'v87', 'v88',
       'v94', 'v95', 'v96', 'v97', 'v103', 'v104', 'v106', 'v115', 'v121',
       'v122', 'v126', 'v131'],
      dtype='object')

In [21]:
X_train_selected = sel_.transform(X_train.fillna(0))
X_test_selected = sel_.transform(X_test.fillna(0))
X_train_selected.shape, X_test_selected.shape

((35000, 80), (15000, 80))

In [22]:
# Separating the data into train and test set 
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target', 'ID'], axis=1),
    data['target'],
    test_size=0.3,
    random_state=0)
X_train.shape, X_test.shape

((35000, 112), (15000, 112))

In [23]:
l1_logit = LogisticRegression(C=1, penalty='l2')
l1_logit.fit(scaler.transform(X_train.fillna(0)), y_train)

/home/simon2/.virtualenvs/ev3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
np.sum(l1_logit.coef_ == 0)

0